$\color{red}{\text{Tried with provided API, but the key was expired}}$


In [23]:
    for i in range(20):

        nurl = f"https://maps.googleapis.com/maps/api/place/textsearch/json?pagetoken={next_page_token}&key=mykey"      
        npayload={}
        nheaders = {}
        nresponse = requests.request("GET", nurl, headers=nheaders, data=npayload)
        next_page_token=nresponse.json()['next_page_token']

        print(nresponse.text)

{
   "html_attributions" : [],
   "next_page_token" : "Aap_uECmhSWar2LsQiYthpj29ILSqnLK5H7XxGTvH92LsslcEla4CgaJH9JjBu98yqzcfrGf1VrHzkmaZSTf3Pn6jHuXTOG9fPbrGiskIQCl1izznkZBCvWPOEIALERlWa3_3KKDZg5j6kQeC3t3AsmOrjlSH4guci8JPvnOFfaXpamhhVSBAc9DFpB673L9tuUpXFVHAQTkrqPIf5kjn9XQrUbS7DHqfCDCYPRtAtuSeUQuaxw_kKrXvAlRA1t3AgY9ZlDvBxaRH1zWxCAV6vZjFvOu-Pr8vSTNs2xfE84ZbbOf1HUii-ScZenbbfzVWynQtshmQBcLTfnFl3svBdGC90q7ssecADudLPhzn9lm2J4N5p8tdFJTOUWVU_lR8dQiA0EXjqK_G0qPOtidiu6dc2nkbGP2wqq87rh6IqzcVcP6FNYPPw9QtKjZRDsnZ4gK-s0-dYR7Dx0aDvlks3hbQyu1BF5KDPtAwpY6bnlJASiw0KEJQTVTt6DmH8CN-JwSIrT2xjZCWTwzK872rT-UVSEzG5_XB8a8UK4NruppdgJACleK6nHecskzzFKyj2YMvac0DNbVt_vebZYI_mRgsU7n7oczlHDgwzvvKLpl9ua-vz87-CTL",
   "results" : [
      {
         "business_status" : "OPERATIONAL",
         "formatted_address" : "1692 Golden Plaza (2nd & 3rd Floor GEC Circle, CDA Ave, Chattogram 4000, Bangladesh",
         "geometry" : {
            "location" : {
               "lat" : 22.3607029,
               "lng" : 91.82135819999

KeyError: 'next_page_token'

<h2>$\color{green}{\text{Importing Libraries}}$</h2>


In [90]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from selenium.webdriver.common.touch_actions import TouchActions
from selenium.webdriver.common.keys import Keys 


import io
import csv
import json
import requests
from bs4 import BeautifulSoup
import time

import pandas as pd
import numpy as np



<h3>$\color{black}{\text{Waiting time between operations & misc.}}$</h3>


In [113]:
#Waiting time between operations 

startup_delay=5
next_page_delay = 3.5
wait_after_element_loaded= 2
delay_to_check_if_element_visible = 1
load_specific_result_delay = 4

csv_file = "Restaurants_in_Bangladesh.csv"
csv_columns = ['name','rating','total_rating_given','price_level','address']

map_query= "https://www.google.com/maps/search/restaurants+in+bangladesh/&pws=0"

<h3>$\color{blue}{\text{Defining extractor functions.}}$</h3>


In [130]:
table=[]
def google_maps_extractor(driver,url):
#     driver.get('chrome://settings/')
#     driver.execute_script('chrome.settingsPrivate.setDefaultZoom(0.50);')
    driver.get(url)

    
    #Intializing with the assumption that there are more than one page result
  

    more_pages = True
    master_restaurant_array = []
    ctr = 0
    
    while more_pages:
        time.sleep(next_page_delay) #takes_few_seconds_to_go_next_page
        

        html = driver.page_source
        
        divs = WebDriverWait(driver,3).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,"div"))) #wait_till_loading
        
        soup = BeautifulSoup(html,'lxml')
        
        time.sleep(wait_after_element_loaded)

        result_list = soup.find_all('div',attrs={'class': 'V0h1Ob-haAclf OPZbO-KE6vqe o0s21d-HiaYvf'})
        #         result_list_span = soup.find_all('span',attrs={'class': 'ZkP5Je'})
#         result_scroll_panel = driver.find_element_by_css_selector("div[class^='siAUzd-neVct section-scrollbox cYB2Ge-oHo7ed cYB2Ge-ti6hGc siAUzd-neVct-Q3DXx-BvBYQ']")
        
        
        for result_val in result_list:
            
            cleaned_data={}   #Empty_dictionary
          


            div = result_val.find("div", class_="qBF1Pd gm2-subtitle-alt-1")
            name= div.span.text
          
    
            rating= result_val.find('span',class_='MW4etd')
        
            if rating !=None:
                rating = rating.text
            else:
                rating='0'
            total_rating_given= result_val.find('span',class_='UY7F9')
            
            #checking if any info is not provided : 'None'
            
            if total_rating_given !=None:
                total_rating_given=total_rating_given.text
            else:
                total_rating_given ='0'
            
            price_level=result_val.find('span', attrs={'jsan':"0.aria-label,0.role"})

            if price_level !=None:
                price_level=price_level.text
            else:
                price_level = "not specified"
            referenceSpan= result_val.find('span',class_='bXlT7b-hgDUwe')
            if referenceSpan!=None:
                address = referenceSpan.find_next_sibling('span')

            else:
                address = None
            
            
            if address!=None:
                address=address.text.strip()
            else:
                address="Not Provided"



            cleaned_data["name"] = name
            cleaned_data["rating"] = rating.replace("(", "").replace(")","").replace("-","").strip()        
            cleaned_data["total_rating_given"] = total_rating_given.replace("(", "").replace(")","").replace("-","").strip()          
            cleaned_data["price_level"] = price_level
            cleaned_data["address"] = address


            print(f"******Name:{name}********")
            print(f"rating:{rating}")
            print(f"total_rating_given:{total_rating_given}")
            print(f"price_level:{price_level}")
            print(f"address:{address}")
            print("")
                                 
            
#             try:
#                 with open(csv_file, 'a', encoding='utf-8') as csvfile:
#                     writer = csv.DictWriter(csvfile, fieldnames=csv_columns, delimiter='|')
                    
#                     writer.writerow(cleaned_data)
#             except IOError:
#                 print("I/O Error")
                
            
#              ActionChains(driver).click(back_btn).perform()
#              time.sleep(back_to_main_list_delay)
            
            table.append(cleaned_data)
     
        next_btn = driver.find_element_by_xpath('//*[contains(@id, "ppdPk-Ej1Yeb-LgbsSe-tJiF1e")]')
        
        if next_btn is None:
            more_pages=False
            break
        time.sleep(next_page_delay)
        ActionChains(driver).click(next_btn).perform()
        

                

In [42]:
import warnings
warnings.filterwarnings('ignore')


In [118]:
options = webdriver.ChromeOptions()
options.add_argument('--lang=en-GB') #Setting language to En-UK

driver = webdriver.Chrome(chrome_options=options)
time.sleep(startup_delay)
google_maps_extractor(driver,map_query,)

******Name:বনানী হোটেল & রেস্টুরেন্ট********
rating:4.2
total_rating_given:(14)
price_level:not specified
address:315 CDA Ave

******Name:Ambrosia Restaurant Ltd********
rating:4.2
total_rating_given:(2,068)
price_level:$$
address:1053, Ground Floor & 1st Floor, Jiban Bima Bhaban, Sheikh Mujib Rd

******Name:THE KABABIAN Multi-Cuisine RESTAURANT********
rating:4.1
total_rating_given:(198)
price_level:not specified
address:House # 7 ( opposite Abahoni Club), Road # 1 (AC Mosjid Road),, Block # G Halishahar H/Estate Chittagong

******Name:Khandani Restaurant********
rating:3.6
total_rating_given:(584)
price_level:$$
address:12/A, 3rd Floor, Shahamanat Complex Near Qrex Diagnostic Center, PC Road

******Name:Handi Restaurant********
rating:4.0
total_rating_given:(5,018)
price_level:$$
address:M M Ali Road, Punak Bhaban, CDA Ave

******Name:bonjour Buffet Lunch & Dinner Everyday********
rating:4.5
total_rating_given:(64)
price_level:not specified
address:Golden Plaza, 1692 CDA Ave

******N

******Name:Sandip vhat Ghar********
rating:3.7
total_rating_given:(27)
price_level:$$
address:Artilari Road, Halishahor

******Name:Pubali Restaurant & Birani House********
rating:0
total_rating_given:0
price_level:$
address:285, Puran Girja, West of Laldighi

******Name:Royal Hotel And Restaurant********
rating:3.6
total_rating_given:(347)
price_level:$$
address:Agrabad Access Road, Agrabad, Barapole

******Name:Dream Brakers Factory********
rating:4.1
total_rating_given:(10)
price_level:$$
address:Road Number 2

******Name:Royal Hut********
rating:4.2
total_rating_given:(2,027)
price_level:$$
address:15, O.R Nizam Road, Panchlaish, Opp. Shishu Park M.A Aziz Stadium Market

******Name:Bangla Garden********
rating:0
total_rating_given:0
price_level:$$
address:Dhaka - Chittagong Hwy

******Name:Cafe Spice********
rating:4.1
total_rating_given:(25)
price_level:not specified
address:Golondaz Rd

******Name:Mohora Restaurent********
rating:3.9
total_rating_given:(17)
price_level:not specif

******Name:Aqua Deck , InterContinental Dhaka********
rating:4.5
total_rating_given:(24)
price_level:$$
address:1 Minto Rd

******Name:Xindian Restaurant********
rating:4.0
total_rating_given:(3,610)
price_level:$$
address:House No.55, 55/A (New), 273 (Old) Road No.: 16 (New), 27(Old, 55/1 Rd No. 27

******Name:Mainland China********
rating:4.2
total_rating_given:(1,115)
price_level:$$$
address:Plot No 15, 14th Floor, Siaaam Tower, Service Rd

******Name:Beauty Lacchi and Faluda********
rating:4.3
total_rating_given:(5,535)
price_level:$
address:61, 1 Kazi Alauddin Rd

******Name:S'MORES CAFE & RESTAURANT********
rating:4.0
total_rating_given:(889)
price_level:$$
address:28, 20 Road No. 2

******Name:Pizza Hut********
rating:4.3
total_rating_given:(6,542)
price_level:$$
address:Plot # 754, Satmasjid Road

******Name:Dhaba********
rating:4.1
total_rating_given:(500)
price_level:$
address:House-15 Lane-14

******Name:A & W********
rating:4.0
total_rating_given:(1,284)
price_level:$$
addr

******Name:Butterfly Park Bangladesh********
rating:3.9
total_rating_given:(1,526)
price_level:$$
address:15, Naval Academy Road, Beside Airport, Patenga

******Name:Sajna Art Gallery********
rating:5.0
total_rating_given:(1)
price_level:$$
address:B-60, Gulshan Shopping Center, Bir Uttom Mir Showkat Sarak, Gulshan-1

******Name:HungryNaki********
rating:3.6
total_rating_given:(266)
price_level:$$
address:Asfia Tower, House - 76, Rd No. 11

******Name:Hotel Pacific********
rating:3.7
total_rating_given:(372)
price_level:$$
address:Not Provided

******Name:Butterfly Park Bangladesh********
rating:3.9
total_rating_given:(1,526)
price_level:$$
address:15, Naval Academy Road, Beside Airport, Patenga

******Name:Sajna Art Gallery********
rating:5.0
total_rating_given:(1)
price_level:$$
address:B-60, Gulshan Shopping Center, Bir Uttom Mir Showkat Sarak, Gulshan-1

******Name:HungryNaki********
rating:3.6
total_rating_given:(266)
price_level:$$
address:Asfia Tower, House - 76, Rd No. 11

****

KeyboardInterrupt: 

In [119]:
df = pd.DataFrame(data=table)
df.to_csv("wfp_programs.csv", encoding='utf-8-sig')

In [121]:
df

,name,rating,total_rating_given,price_level,address
0,বনানী হোটেল & রেস্টুরেন্ট,4.2,14,not specified,315 CDA Ave
1,Ambrosia Restaurant Ltd,4.2,"2,068",$$,"1053, Ground Floor & 1st Floor, Jiban Bima Bha..."
2,THE KABABIAN Multi-Cuisine RESTAURANT,4.1,198,not specified,"House # 7 ( opposite Abahoni Club), Road # 1 (..."
3,Khandani Restaurant,3.6,584,$$,"12/A, 3rd Floor, Shahamanat Complex Near Qrex ..."
4,Handi Restaurant,4.0,"5,018",$$,"M M Ali Road, Punak Bhaban, CDA Ave"
...,...,...,...,...,...
225,Hotel Pacific,3.7,372,$$,Not Provided
226,Butterfly Park Bangladesh,3.9,"1,526",$$,"15, Naval Academy Road, Beside Airport, Patenga"
227,Sajna Art Gallery,5.0,1,$$,"B-60, Gulshan Shopping Center, Bir Uttom Mir S..."
228,HungryNaki,3.6,266,$$,"Asfia Tower, House - 76, Rd No. 11"


In [122]:
import googlemaps


In [125]:
gmaps_key = googlemaps.Client(key = "my_key")


$\color{blue}{\text{Geocoding the scapped locations.}}$


In [ ]:
df["lat"] = None
df["lon"] = None


for i in range(0, len(df), 1):
    geocode_result = gmaps_key.geocode(df.iat[i,0])
    try:
        lat = geocode_result[0]["geometry"]["location"]["lat"]
        lon = geocode_result[0]["geometry"]["location"]["lon"]
        df.iat[i, df.cloumns.get_loc("lat")]=lat
        df.iat[i, df.cloumns.get_loc("lon")]=long
    except:
        lat= None
        lon=None

<h1>$\color{green}{\text{CSV file is uploaded in the repisotory.}}$<h1>